## **BAY AREA 2019**

# Extracción

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Definir la URL para Bay Area 2019 (San Francisco y San José) - repositorio público AWS
url = 'https://s3.amazonaws.com/baywheels-data/'

# Se solicita la información a la página y usamos BeautifulSoup para obtenre un listado estructurado en XML y extraemos los archivos .zip de 2019
response = requests.get(url)
files_2019 = [
    file.text for file in BeautifulSoup(response.text, 'xml').find_all('Key')
    if file.text.endswith('.zip') and '2019' in file.text
]

print(f"Encontrados {len(files_2019)} archivos para Bay Area 2019")

Encontrados 12 archivos para Bay Area 2019


# Transformaciones

### Transformación 1: Filtrado de columnas y transformación 2: Creación campo ciudad

In [2]:
# T.1 TRANSFORMACIÓN DE FILTRADO: definimos el nombre del archivo CSV final y las columnas requeridas.
output_file = 'BayArea_2019.csv'
required_columns = ['start_time', 'end_time', 'start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude', 'user_type'] #filtramos por las 7 columnas requeridas


In [3]:
import os
import zipfile

# Verificamos si el archivo de salida ya existe para no sobreescribirlo
if not os.path.exists(output_file):
    # Crear el archivo CSV con la columnas requeridas y la columna adicional 'city'
    pd.DataFrame(columns=required_columns + ['city']).to_csv(output_file, index=False)

# Iteramos sobre cada archivo mensual de 2019
for file_name in files_2019:
    print(f"Procesando archivo: {file_name}")

    # Descargar el archivo y descomprimir
    file_url = url + file_name
    local_zip = file_name.split('/')[-1] #nos quedamos con el nombre del mes del archivo en lugar de toda la dirección URL

    with open(local_zip, "wb") as f: #se crea un archivo vacio para meter la información de la URL de cada mes
        f.write(requests.get(file_url).content)

    # Descomprimir el archivo en una carpeta temporal
    with zipfile.ZipFile(local_zip, "r") as zip_ref: #se abre el archivo ZIP para leerlo
        zip_ref.extractall("temp_data") #se extrae la información y se añade en la carpeta temporal

    # Cargar el CSV descomprimido y filtrar las columnas necesarias
    csv_file = [file for file in os.listdir("temp_data") if file.endswith('.csv')][0]
    df = pd.read_csv(os.path.join("temp_data", csv_file), usecols=required_columns)

    # T.2 TRANSFORMACIÓN CREACIÓN CAMPO CIUDAD: añadir la columna 'city' con el valor "San Francisco y San Jose" en todas las filas
    df['city'] = 'San Francisco & San Jose'

    # Guardar los datos en el archivo CSV final (añadir al archivo existente)
    df.to_csv(output_file, mode='a', index=False, header=False)

    # Limpiar archivos temporales utilizados durante este apartado
    os.remove(local_zip)
    os.remove(os.path.join("temp_data", csv_file))

print(f"Archivo '{output_file}' actualizado con los meses de 2019 y la columna 'city' añadida.")


Procesando archivo: 201901-fordgobike-tripdata.csv.zip
Procesando archivo: 201902-fordgobike-tripdata.csv.zip
Procesando archivo: 201903-fordgobike-tripdata.csv.zip
Procesando archivo: 201904-fordgobike-tripdata.csv.zip
Procesando archivo: 201905-baywheels-tripdata.csv.zip
Procesando archivo: 201906-baywheels-tripdata.csv.zip
Procesando archivo: 201907-baywheels-tripdata.csv.zip
Procesando archivo: 201908-baywheels-tripdata.csv.zip
Procesando archivo: 201909-baywheels-tripdata.csv.zip
Procesando archivo: 201910-baywheels-tripdata.csv.zip
Procesando archivo: 201911-baywheels-tripdata.csv.zip
Procesando archivo: 201912-baywheels-tripdata.csv.zip
Archivo 'BayArea_2019.csv' actualizado con los meses de 2019 y la columna 'city' añadida.


In [4]:
# Imprimimos el df para comprobar su estructura y el formato de los datos
df = pd.read_csv('BayArea_2019.csv')
df

# Se puede observar que el formato fecha conserva la información de los milisegundos en las primeras filas. Habrá que transformarlo para que tenga el formato estándar elegido (hasta segundos)

,start_time,end_time,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,user_type,city
0,2019-01-31 17:57:44.6130,2019-02-01 16:24:49.8640,37.775745,-122.213037,37.808894,-122.256460,Subscriber,San Francisco & San Jose
1,2019-01-31 20:58:33.8860,2019-02-01 15:16:54.1730,37.785881,-122.408915,37.752428,-122.420628,Subscriber,San Francisco & San Jose
2,2019-01-31 18:06:52.9240,2019-02-01 11:30:46.5300,37.870139,-122.268422,37.846784,-122.291376,Customer,San Francisco & San Jose
3,2019-01-31 19:46:09.7190,2019-02-01 08:10:50.3180,37.770083,-122.429156,37.775946,-122.437777,Customer,San Francisco & San Jose
4,2019-01-31 14:19:01.5410,2019-02-01 07:10:51.0650,37.794130,-122.394430,37.787168,-122.388098,Customer,San Francisco & San Jose
...,...,...,...,...,...,...,...,...
2506978,2019-12-28 18:37:29,2019-12-28 18:49:40,37.809731,-122.409855,37.807573,-122.421681,Subscriber,San Francisco & San Jose
2506979,2019-12-28 18:37:24,2019-12-28 18:49:53,37.809768,-122.409797,37.807558,-122.421638,Subscriber,San Francisco & San Jose
2506980,2019-12-26 14:25:37,2019-12-26 14:43:02,37.810183,-122.410271,37.807225,-122.408850,Subscriber,San Francisco & San Jose
2506981,2019-12-30 20:35:43,2019-12-30 20:41:09,37.810471,-122.417386,37.803143,-122.414373,Subscriber,San Francisco & San Jose


### Transformación 4: Corrección formato fecha

In [5]:
# Reemplaza los milisegundos por vacio, solo cuando están presentes (buscando cualquier dato que tenga un punto seguido de números, esto son los milisegundos)
for col in ['start_time', 'end_time']:
    df[col] = df[col].str.replace(r'\.\d+$', '', regex=True) #busca en las columnas los puntos (\.) seguidos de números (\d) y selecciona el final de la cadena (+$)

# Verificación rápida
df[['start_time', 'end_time']].head()

,start_time,end_time
0,2019-01-31 17:57:44,2019-02-01 16:24:49
1,2019-01-31 20:58:33,2019-02-01 15:16:54
2,2019-01-31 18:06:52,2019-02-01 11:30:46
3,2019-01-31 19:46:09,2019-02-01 08:10:50
4,2019-01-31 14:19:01,2019-02-01 07:10:51


In [6]:
# Guardar el dataframe en csv
print(f"El archivo Bay Area 2019 tiene {df.shape[0]} filas y {df.shape[1]} columnas")
df.to_csv('BayArea_2019.csv', index=False)

El archivo Bay Area 2019 tiene 2506983 filas y 8 columnas
